In [1]:
library(httr)

In [131]:
github_api <- function(path,per_page,page) {
    
    if(!is.numeric(per_page)){
        stop("The per_page parameter should not be non-numerical value")
    }
    
    if(!is.numeric(page)){
        stop("The page parameter should not be non-numerical value")
    }
    
    if (per_page < 1 || per_page > 100 || per_page != round(per_page)) {
        stop("The per_page parameter should be a integer between 1 and 100 (inclusive)", call. = FALSE)
    }
    
    if (page < 1 || page != round(page)) {
        stop("The page parameter should be a positive integer", call. = FALSE)
    }
    
    query <- list(per_page=per_page,page=page)
    
    url <- modify_url("https://api.github.com", path = path)
    response <- GET(url, add_headers(accept = "application/vnd.github.v3+json"), query = query, authenticate("username", "token"))
    
    if (http_error(response)) {
        stop("GitHub API request failed", call. = FALSE)
    }
    
    if (http_type(response) != "application/json") {
        stop("API did not return json", call. = FALSE)
    }
    response
}

format_response <- function(response,output_type) {
    result <- NULL
    if (output_type == "list") {
        result <- jsonlite::fromJSON(content(response, "text"), simplifyVector = FALSE)
    } else if (output_type == "dataframe") {
        result <- jsonlite::fromJSON(rawToChar(response$content),flatten = TRUE)
    } else if (output_type == "character") {
        result <- rawToChar(response$content)
    } else if (output_type == "raw") {
        result <- response
    } else {
        stop("Unexpected output type, possible values are: 'list', 'dataframe', 'character'")
    }
    
    result
}

In [34]:
# resp <- github_api("/events",list(per_page=1,page=0))
# resp

In [48]:
github_get_public_events <- function(per_page=30, page=1, output_type="list"){
    
    response <- github_api("/events",per_page,page)
    
    result <- format_response(response,output_type)
    
    result
}

In [142]:
# github_get_public_events()

In [62]:
github_get_organization_events <- function(organization, per_page=30, page=1, output_type="list"){
        
    path <- paste("/orgs/", organization, "/events", sep="")

    response <- github_api(path,per_page,page)
    
    result <- format_response(response,output_type)
    
    result
}

In [66]:
# github_get_organization_events("ubco-mds-2021",3,1,"dataframe")

In [65]:
# github_get_organization_events("ubco-mds-2021")

In [69]:
github_get_network_events <- function(owner, repo, per_page=30, page=1, output_type="list"){
        
    path <- paste("/networks/", owner, "/", repo, "/events", sep="")

    response <- github_api(path,per_page,page)
    
    result <- format_response(response,output_type)
    
    result
}

In [71]:
# github_get_network_events("AndrewNg1891","DATA534_API_Project_Group9")

In [80]:
github_get_repo_events <- function(owner, repo, per_page=30, page=1, output_type="list"){
        
    path <- paste("/repos/", owner, "/", repo, "/events", sep="")

    response <- github_api(path,per_page,page)
    
    result <- format_response(response,output_type)
    
    result
}

In [85]:
aa <- github_get_repo_events("AndrewNg1891","DATA534_API_Project_Group9",100,2,"dataframe")

In [86]:
length(aa)

[1] 0

In [87]:
aa

list()

In [138]:
github_count_events_bytype <- function(range, max_event_num){
    if (max_event_num < 1 || max_event_num != round(max_event_num)) {
        stop("The max_event_num parameter should be a positive integer", call. = FALSE)
    }
    total_pages <- ceiling(max_event_num/100)
    print(total_pages)
    result <- NULL
    datalist = list()
    for (i in 1:total_pages){
        if (range == "all public") {
            temp_df <- github_get_public_events(100, page=i,"dataframe")
            temp_df <- temp_df[,c("id","type","created_at")]
        }
        
        if (length(temp_df) > 0) {
            datalist[[i]] <- temp_df
        } else {
            break
        }
    }
    
    binded_df <- dplyr::bind_rows(datalist)
    binded_df <- binded_df[1:max_event_num,]
    print(dplyr::dim_desc(binded_df))
    result <- binded_df |> dplyr::count(type, name = "count")
    result
    
}

In [139]:
testit <- github_count_events_bytype("all public",240)

[1] 3
[1] "[240 x 3]"


In [140]:
testit

type,count
<chr>,<int>
CommitCommentEvent,1
CreateEvent,37
DeleteEvent,9
ForkEvent,4
IssueCommentEvent,7
IssuesEvent,7
MemberEvent,2
PublicEvent,2
PullRequestEvent,16


In [141]:
sum(testit[,"count"])

[1] 240